In [ ]:
import pandas as pd

df = pd.read_csv('ARC_trials_v3_mri.csv',index_col=False)
df=pd.DataFrame(df)

trial_1 = df.iloc[0:81]
trial_2 = df.iloc[81:162]
trial_3 = df.iloc[162::]

block1_time_sec = sum(trial_1['ITI']+(5.25*81))
block1_time_min = (sum(trial_1['ITI'])+(5.25*81))/60

block2_time_sec = sum(trial_2['ITI'])+(5.25*81)
block2_time_min = (sum(trial_2['ITI'])+(5.25*81))/60

block3_time_sec = sum(trial_3['ITI'])+(5.25*81)
block3_time_min = (sum(trial_3['ITI'])+(5.25*81))/60


print 'Block 1 | ITI sum: %s' % sum(trial_1['ITI'])
print '        | Total Block Time: %s sec (%s minutes)' % (block1_time_sec,block1_time_min)
print '        | Risk sum: %s' % sum(trial_1['Risk'])
print '        | reward sum: %s \n' % sum(trial_1['Reward'])

print 'Block 2 | ITI sum: %s' % sum(trial_2['ITI'])
print '        | Total Block Time: %s sec (%s minutes)' % (block2_time_sec,block2_time_min)
print '        | Risk sum: %s' % sum(trial_2['Risk'])
print '        | Reward sum: %s \n' % sum(trial_2['Reward'])

print 'Block 3 | ITI sum: %s' % sum(trial_3['ITI'])
print '        | Total Block Time: %s sec (%s minutes)' % (block3_time_sec,block3_time_min)
print '        | Risk sum: %s' % sum(trial_3['Risk'])
print '        | Reward sum: %s' % sum(trial_3['Reward'])


# display(trial_1.head())
# display(trial_2.head())
# display(trial_3.head())

# print trial_1['ITI'].value_counts()

In [5]:
import os
import pandas as pd
import numpy as np
# import tables

dir='/Users/emilyhahn/projects/newARC/behavior/raw'

sub_list=[]
sub_files_list=[]

for root, dirs, files in os.walk(dir, topdown=False):
    for name in files:
        sub_list.append(name)
        sub_files_list.append(name)
        
print sub_files_list
        
# df=pd.DataFrame()

# for sub,csv in zip(sub_list,sub_files_list):
#     d=os.path.join(dir,csv)
#     behavior=pd.read_csv(d)
#     print behavior    

['hc015_ARC_eeg_block_3_2018_07_03_17_18_21.mat', 'hc014_ARC_eeg_block_2_2018_05_09_12_25_19.mat', 'hc013_ARC_eeg_block_1_2018_05_10_18_01_24.mat', 'hc005_ARC_mri_2018_04_22_10_54_02.mat', 'hc005_ARC_mri_2018_04_22_10_56_13.mat', 'hc017_ARC_eeg_block_1_2018_11_16_16_17_08.mat', 'hc003_ARC_eeg_2018_04_07_14_00_49.mat', 'hc014_ARC_mri_block_3_2018_05_06_10_40_37.mat', 'hc014_ARC_eeg_block_3_2018_05_09_12_33_11.mat', 'hc004_ARC_mri_2018_04_08_10_53_57.mat', 'hc010_ARC_eeg_block_2_2018_10_03_18_47_56.mat', 'hc009_ARC_mri_block_1_2018_05_05_10_43_23.mat', 'hc012_ARC_mri_block_1_2018_06_26_20_09_34.mat', 'hc007_ARC_mri_block_2_2018_04_28_10_01_54.mat', 'hc013_ARC_mri_block_3_2018_05_08_20_34_01.mat', 'hc019_ARC_mri_block_2_2018_12_04_20_52_02.mat', 'hc014_ARC_mri_block_2_2018_05_06_10_28_56.mat', 'hc013_ARC_mri_block_1_2018_05_08_19_59_37.mat', 'hc015_ARC_eeg_block_2_2018_07_03_17_11_21.mat', 'hc015_ARC_mri_block_2_2018_07_03_20_09_08.mat', 'hc012_ARC_eeg_block_1_2018_07_06_10_04_52.mat', 'h

In [1]:
import os
import pandas as pd
import numpy as np
# import tables

dir='/Users/emilyhahn/projects/newARC/behavior/preprocessed'

sub_list=[]
sub_files_list=[]

for root, dirs, files in os.walk(dir, topdown=False):
    for name in files:
        sub_list.append(name[:5])
        sub_files_list.append(name)

for sub,csv in zip(sub_list,sub_files_list):
    print sub
#     d=os.path.join(dir,csv)
#     df=pd.read_csv(d)
#     print df['ConflictType']